<a href="https://colab.research.google.com/github/KornelWitkowski/TensorFlow-benchmarks/blob/main/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow_datasets as tfds

dataset_goemotions = tfds.load("goemotions")

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/goemotions/0.1.0.incompleteBHZGTM/goemotions-train.tfrecord


  0%|          | 0/43410 [00:00<?, ? examples/s]

...

  0%|          | 0/5426 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/goemotions/0.1.0.incompleteBHZGTM/goemotions-test.tfrecord


  0%|          | 0/5427 [00:00<?, ? examples/s]

Dataset goemotions downloaded and prepared to /root/tensorflow_datasets/goemotions/0.1.0. Subsequent calls will reuse this data.


In [3]:
test_data = dataset_goemotions["test"]
train_data = dataset_goemotions["train"]
validation_data = dataset_goemotions["validation"]

In [4]:
len(train_data), len(test_data), len(validation_data)

(43410, 5427, 5426)

In [5]:
test_data = tfds.as_dataframe(test_data)
train_data = tfds.as_dataframe(train_data)
validation_data = tfds.as_dataframe(validation_data)

In [6]:
test_data.head()

,admiration,amusement,anger,annoyance,approval,caring,comment_text,confusion,curiosity,desire,...,love,nervousness,neutral,optimism,pride,realization,relief,remorse,sadness,surprise
0,False,False,False,False,True,False,"b""You're right, thanks for pointing that out, ...",False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,b'Molon labe!!!',False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,b'So this is what edging feels like',False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,b'Just speak with a Sheffield accent lmao',False,False,False,...,False,False,True,False,False,False,False,False,False,False
4,False,False,False,False,False,False,b'My husband and I have been circling the drai...,False,False,False,...,False,False,False,False,False,False,False,False,True,False


In [10]:
class_names = list(test_data.drop("comment_text", axis=1).columns)
print(class_names)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise']


In [11]:
import re

strip = lambda x: re.findall(r"b['\"](.+)['\"]", x)[0]
replace = lambda x: x.replace('\\xe2\\x80\\x99', "'")

def process_datafeame(df):

  df_labels = df.drop("comment_text", axis=1).astype(int)
  df_sentences = df["comment_text"].astype('string')

  df_sentences = df_sentences.map(strip)
  df_sentences = df_sentences.map(replace)

  return df_sentences, df_labels

In [12]:
train_sentences, train_labels = process_datafeame(test_data)
test_sentences, test_labels = process_datafeame(train_data)
valid_sentences, valid_labels = process_datafeame(validation_data)

valid_sentences.head(10)

0    Wait, I see the problem, you are changing the ...
1             We are there right now. \xf0\x9f\x8d\xba
2    I usually do. I didn't stay up until midnight ...
3    Just as delusional as [NAME] was. The differen...
4    Did you miss the massive throbbing erection th...
5    Probably because every day several people ask ...
6    Bad execution but it made me smile through the...
7    Are you willing to donate to local organizatio...
8                                Happy birtday my dude
9    Are they bigger than our defensive breakdowns ...
Name: comment_text, dtype: object

In [36]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(train_sentences),
                                              tf.convert_to_tensor(train_labels)))

valid_ds = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(valid_sentences),
                                              tf.convert_to_tensor(valid_labels)))

test_ds = tf.data.Dataset.from_tensor_slices((tf.convert_to_tensor(test_sentences),
                                              tf.convert_to_tensor(test_labels)))

train_ds = train_ds.batch(128).shuffle(1000).prefetch(tf.data.AUTOTUNE)
valid_ds = valid_ds.batch(128).shuffle(1000).prefetch(tf.data.AUTOTUNE)
test_ds  = test_ds.batch(128).shuffle(1000).prefetch(tf.data.AUTOTUNE)

In [23]:
avg_words = 0
for sentence in test_sentences:
  avg_words += len(sentence.split())

avg_words/len(test_sentences)

12.840175074867542

In [26]:
from tensorflow.keras.layers import TextVectorization

vocab_length = 20000
sentence_length = 20

text_vectorizer = TextVectorization(max_tokens=vocab_length,
                                    output_mode="int",
                                    output_sequence_length=sentence_length)

In [27]:
text_vectorizer.adapt(train_sentences)

In [28]:
sample_sentence = "A an the I you she he it play like would purple crimson"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 20), dtype=int64, numpy=
array([[   5,   68,    2,    3,    6,   81,   35,   10,  258,   29,   54,
        5263,    1,    0,    0,    0,    0,    0,    0,    0]])>

In [29]:
import random

random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
What's does the future hold for you two? You're not happy with him. You deserve better.      

Vectorized version:


<tf.Tensor: shape=(1, 20), dtype=int64, numpy=
array([[260, 164,   2, 454, 963,  14,   6, 355,  66,  16, 142,  24,  69,
          6, 457, 124,   0,   0,   0,   0]])>

In [30]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_10_words = words_in_vocab[:10] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_10_words = words_in_vocab[-10:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 10 most common words: {top_10_words}") 

Number of words in vocab: 8911
Top 10 most common words: ['', '[UNK]', 'the', 'i', 'to', 'a', 'you', 'and', 'is', 'name']


In [31]:
tf.random.set_seed(42)
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim=len(words_in_vocab), # set input shape
                      output_dim=64, # set size of embedding vector
                      input_length=sentence_length, # how long is each input
                      name="embedding_1") 

In [32]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Thanks. Appreciate that perspective. I\'m just saying it as a counter to the "rope is the only cope" mentality on the incel subreddit.      

Embedded version:


<tf.Tensor: shape=(1, 20, 64), dtype=float32, numpy=
array([[[ 0.01519776,  0.02454609,  0.03962648, ...,  0.03516266,
          0.0425827 ,  0.04859931],
        [ 0.03095556,  0.00029144,  0.03964074, ..., -0.04924453,
         -0.04235966, -0.01974863],
        [-0.02853444, -0.00442432,  0.03031475, ..., -0.04792733,
         -0.01664252, -0.03293632],
        ...,
        [ 0.0298998 , -0.04326692,  0.03723611, ...,  0.01994223,
          0.01815477, -0.01751149],
        [-0.04758058, -0.03839898, -0.00203277, ...,  0.01275707,
         -0.03692679,  0.04111798],
        [ 0.01438809,  0.04091888, -0.01207281, ..., -0.01858177,
         -0.04566612,  0.04470313]]], dtype=float32)>

In [38]:
# Build model with the Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(128, return_sequences=True, dropout=0.2)(x)
x = layers.LSTM(128, return_sequences=True, dropout=0.2)(x)
x = layers.LSTM(64, dropout=0.2)(x)
x = layers.Dense(64, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(28, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                metrics=["accuracy"])

# Get a summary of the model
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 20)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 20, 64)            570304    
                                                                 
 lstm_3 (LSTM)               (None, 20, 128)           98816     
                                                                 
 lstm_4 (LSTM)               (None, 20, 128)           131584    
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                     

In [39]:
import os

def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                            verbose=0,
                                            monitor="val_accuracy",
                                            save_best_only=True) # save only the best model to file

In [40]:
model_1_history = model_1.fit(train_ds,
                              epochs=10,
                              validation_data=valid_ds,
                              callbacks=[create_model_checkpoint(model_1.name)])

Epoch 1/10
39/43 [==========================>...] - ETA: 0s - loss: 0.2130 - accuracy: 0.2358

INFO:tensorflow:Assets written to: model_experiments/model_1_dense/assets


INFO:tensorflow:Assets written to: model_experiments/model_1_dense/assets


43/43 [==============================] - 18s 326ms/step - loss: 0.2076 - accuracy: 0.2414 - val_loss: 0.1499 - val_accuracy: 0.2960
Epoch 2/10
43/43 [==============================] - 1s 18ms/step - loss: 0.1542 - accuracy: 0.3000 - val_loss: 0.1497 - val_accuracy: 0.2960
Epoch 3/10
43/43 [==============================] - 1s 18ms/step - loss: 0.1536 - accuracy: 0.3002 - val_loss: 0.1498 - val_accuracy: 0.2960
Epoch 4/10
43/43 [==============================] - 1s 17ms/step - loss: 0.1524 - accuracy: 0.3004 - val_loss: 0.1496 - val_accuracy: 0.2960
Epoch 5/10
43/43 [==============================] - 1s 18ms/step - loss: 0.1526 - accuracy: 0.3004 - val_loss: 0.1503 - val_accuracy: 0.2960
Epoch 6/10
43/43 [==============================] - 1s 18ms/step - loss: 0.1525 - accuracy: 0.3004 - val_loss: 0.1499 - val_accuracy: 0.2960
Epoch 7/10
43/43 [==============================] - 1s 18ms/step - loss: 0.1520 - accuracy: 0.3004 - val_loss: 0.1494 - val_accuracy: 0.2960
Epoch 8/10
43/43 [====

In [41]:
tf.random.set_seed(42)

model_5_embedding = layers.Embedding(input_dim=vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=sentence_length,
                                     name="embedding_5")


inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Bidirectional(layers.LSTM(100, return_sequences=True, dropout=0.3))(x)
x = layers.Bidirectional(layers.LSTM(100, dropout=0.3))(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(28, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_Bidirectional")

model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Fit the model (takes longer because of the bidirectional layers)
model_5_history = model_5.fit(train_ds,
                              epochs=10,
                              validation_data=valid_ds,
                              callbacks=[create_model_checkpoint(model_5.name)])

Epoch 1/10
41/43 [===========================>..] - ETA: 0s - loss: 0.2768 - accuracy: 0.1928

INFO:tensorflow:Assets written to: model_experiments/model_5_Bidirectional/assets


INFO:tensorflow:Assets written to: model_experiments/model_5_Bidirectional/assets


43/43 [==============================] - 30s 584ms/step - loss: 0.2708 - accuracy: 0.1985 - val_loss: 0.1511 - val_accuracy: 0.2960
Epoch 2/10
43/43 [==============================] - 1s 23ms/step - loss: 0.1549 - accuracy: 0.2882 - val_loss: 0.1494 - val_accuracy: 0.2960
Epoch 3/10
43/43 [==============================] - 1s 22ms/step - loss: 0.1528 - accuracy: 0.2976 - val_loss: 0.1494 - val_accuracy: 0.2960
Epoch 4/10
43/43 [==============================] - 1s 22ms/step - loss: 0.1520 - accuracy: 0.3000 - val_loss: 0.1494 - val_accuracy: 0.2960
Epoch 5/10
43/43 [==============================] - 1s 22ms/step - loss: 0.1513 - accuracy: 0.3002 - val_loss: 0.1494 - val_accuracy: 0.2960
Epoch 6/10
43/43 [==============================] - 1s 22ms/step - loss: 0.1508 - accuracy: 0.3004 - val_loss: 0.1493 - val_accuracy: 0.2960
Epoch 7/10
43/43 [==============================] - 1s 22ms/step - loss: 0.1504 - accuracy: 0.3005 - val_loss: 0.1492 - val_accuracy: 0.2960
Epoch 8/10
43/43 [====

INFO:tensorflow:Assets written to: model_experiments/model_5_Bidirectional/assets


INFO:tensorflow:Assets written to: model_experiments/model_5_Bidirectional/assets


43/43 [==============================] - 24s 570ms/step - loss: 0.1479 - accuracy: 0.3125 - val_loss: 0.1459 - val_accuracy: 0.3192
Epoch 10/10
43/43 [==============================] - ETA: 0s - loss: 0.1442 - accuracy: 0.3252

INFO:tensorflow:Assets written to: model_experiments/model_5_Bidirectional/assets


INFO:tensorflow:Assets written to: model_experiments/model_5_Bidirectional/assets


43/43 [==============================] - 24s 571ms/step - loss: 0.1442 - accuracy: 0.3252 - val_loss: 0.1438 - val_accuracy: 0.3194


In [ ]:
model.evaluate()